In [217]:
import dgl
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from tqdm.auto import trange

from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.mip_solver import mip_solve
from cloudmanufacturing.validation import objvalue, construct_delta
from cloudmanufacturing.graph import dglgraph
from cloudmanufacturing.graphconv import AttnConvLayer, DotProductDecoder

In [2]:
dataset = read_fatahi_dataset('../data/fatahi.xlsx')
problem = dataset[0]
problem['name']

  0%|          | 0/18 [00:00<?, ?it/s]

'5,10,10-1'

In [3]:
delta, gamma, status, value = mip_solve(problem)
print(f'Problem {problem["name"]}: {value:.2f}, {status}')

Problem 5,10,10-1: 5086.08, OptimizationStatus.OPTIMAL


In [72]:
graph = dglgraph(problem, gamma)
graph

Graph(num_nodes={'o': 30, 's': 10},
      num_edges={('o', 'backward', 'o'): 33, ('o', 'forward', 'o'): 33, ('o', 'os', 's'): 215, ('s', 'so', 'o'): 215, ('s', 'ss', 's'): 100},
      metagraph=[('o', 'o', 'backward'), ('o', 'o', 'forward'), ('o', 's', 'os'), ('s', 'o', 'so'), ('s', 's', 'ss')])

### Энкодер

Cвертка в вершины S
* $h^{s}_{i} = W^s S_i $
* $h^{ss}_{ij} = W^{ss} \left[ S_i \Vert SS_{ij}\right]$
* $h^{os}_{ij} = W^{os} \left[O_i \Vert OS_{ij} \right]$
* $e^{os}_{ij} = \text{LeakyReLU}\left(a^\top\left[ h^{os}_{ij} \Vert h^s_j \right]\right)$
* $ e^{ss}_{ij} = \text{LeakyReLU}\left(a^\top\left[h^{ss}_{ij} \Vert h^s_j \right]\right)$
* $\alpha^{os}_i = \text{Softmax}_{(i,j) \in E_{os}} e^{os}_{ij}$
* $\alpha^{ss}_i = \text{Softmax}_{(i,j) \in E_{ss}} e^{ss}_{ij}$
* $z_j = \sum_{(i,j) \in E_{ss}} \alpha^{ss}_i h_{ij}^{ss} + \sum_{(i,j) \in E_{os}}\alpha^{os}_i h^{os}_{ij} $

свертка в вершины O

* $x_j = W^\text{o}\text{ReLU}\left[\sum_{(i,j) \in E_{oo}} W^\text{in} O_i \Vert W^\text{self} O_j \Vert \sum_{(j,i) \in E_{oo}} W^\text{out} O_i \right]$

### Декодер

Софтмакс на ребрах

$m_{ij} = e^{z_i^\top x_j}$

$p_{ij} = \frac{m_{ij}}{\sum_{(l, j) \in E_{so}} m_{lj}}$


In [171]:
class GNN(nn.Module):
    def __init__(self, ins_dim, ino_dim, out_dim):
        super().__init__()
        self.conv0 = AttnConvLayer(ins_dim, ino_dim, out_dim)
        self.conv1 = AttnConvLayer(out_dim, out_dim, out_dim)
        self.dec = DotProductDecoder()
    
    def forward(self, graph):
        s_feat = graph.ndata['feat']['s']
        o_feat = graph.ndata['feat']['o']
        s_hid, o_hid = self.conv0(graph, s_feat, o_feat)
        s_hid, o_hid = self.conv1(graph, s_hid, o_hid)
        prob = self.dec(g, s_hid, o_hid)
        return prob

In [192]:

model(graph)

tensor([0.1311, 0.1330, 0.1469, 0.1415, 0.1354, 0.1321, 0.1800, 0.1885, 0.1771,
        0.1925, 0.2287, 0.2132, 0.1186, 0.1408, 0.1105, 0.1237, 0.1246, 0.1207,
        0.1141, 0.1471, 0.0971, 0.1392, 0.0916, 0.0990, 0.1172, 0.1030, 0.0973,
        0.1006, 0.1550, 0.1065, 0.0994, 0.1085, 0.1142, 0.1132, 0.1081, 0.1077,
        0.1040, 0.1384, 0.1405, 0.1104, 0.1200, 0.1242, 0.1240, 0.1190, 0.1145,
        0.1474, 0.1453, 0.1127, 0.1258, 0.1207, 0.1150, 0.1114, 0.1120, 0.1571,
        0.1167, 0.1435, 0.1082, 0.1243, 0.1240, 0.1187, 0.1129, 0.1517, 0.1003,
        0.1355, 0.0935, 0.1026, 0.1146, 0.1043, 0.1012, 0.1005, 0.1476, 0.1443,
        0.1070, 0.1161, 0.1256, 0.1226, 0.1170, 0.1131, 0.1543, 0.1262, 0.1389,
        0.1432, 0.1435, 0.1372, 0.1382, 0.1727, 0.1863, 0.1745, 0.1902, 0.2344,
        0.2146, 0.1060, 0.0991, 0.1081, 0.1146, 0.1131, 0.1080, 0.1074, 0.1039,
        0.1397, 0.1419, 0.1092, 0.1237, 0.1242, 0.1189, 0.1198, 0.1133, 0.1489,
        0.1042, 0.1158, 0.1319, 0.1260, 

In [214]:
model = GNN(1, problem['n_operations']+2, 32)
optim = Adam(model.parameters(), lr=0.001)

In [228]:
graph = dglgraph(problem, gamma)
graph.edata['feat'][('o', 'os', 's')][:, 0] = graph.edata['feat'][('o', 'os', 's')][:, 0] / 10
graph.edata['feat'][('s', 'ss', 's')][:] = graph.edata['feat'][('s', 'ss', 's')] / 100
target = graph.edata['target'][('o', 'os','s')][:, 0]
for i in trange(100):
    prob = model(graph)
    loss = F.binary_cross_entropy(prob, target)
    optim.zero_grad()
    loss.backward()
    optim.step()
    print(loss.item())

  0%|          | 0/100 [00:00<?, ?it/s]

0.2527468800544739
0.24897931516170502
0.2450663298368454
0.24073068797588348
0.23627012968063354
0.23161667585372925
0.22654925286769867
0.22131304442882538
0.21590551733970642
0.21018779277801514
0.20440971851348877
0.1985923796892166
0.19270355999469757
0.1870233416557312
0.18160368502140045
0.17657370865345
0.172183558344841
0.16842715442180634
0.16534821689128876
0.16279329359531403
0.16042469441890717
0.15810827910900116
0.15567797422409058
0.15317904949188232
0.1507633775472641
0.14855021238327026
0.14658094942569733
0.1447756588459015
0.1430051177740097
0.14121539890766144
0.13936541974544525
0.1375230848789215
0.13574662804603577
0.13407789170742035
0.13251066207885742
0.1310051530599594
0.12951786816120148
0.12805123627185822
0.12659062445163727
0.12519170343875885
0.12390778213739395
0.12273397296667099
0.12163344770669937
0.1205371841788292
0.11935985833406448
0.11810775846242905
0.11681455373764038
0.1154956966638565
0.11415402591228485
0.11276169121265411
0.11129233986139

RuntimeError: all elements of input should be between 0 and 1

In [218]:
with torch.no_grad():
    prob = model(graph)
    u, v = model.dec.sample(graph, prob)
u, v

(tensor([0, 3, 7, 0, 3, 1, 1, 5, 9, 3, 7, 3, 5, 7, 3, 7, 3, 7, 3, 6, 3, 6, 3, 7,
         0, 2, 3, 7]),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27]))